In [1]:
import os
import shutil
import librosa
import numpy as np
from scipy.io import wavfile
import soundfile as sf
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np
from multiprocessing import Pool

# Processing CMU

In [ ]:
os.makedirs('./TestData',exist_ok=True)
os.makedirs('./TrainData',exist_ok = True)

In [ ]:
os.makedirs('./TestData/CMU',exist_ok=True)
os.makedirs('./TrainData/CMU',exist_ok = True)
for drt in ['./cmu_us_bdl_arctic/orig/','./cmu_us_jmk_arctic/orig/','./cmu_us_slt_arctic/orig/']:
    files = os.listdir(drt)
    for file in files:
        if file[7]=='a':
            shutil.copy2(drt+file, './TrainData/CMU/'+drt[9:13]+file)
        if file[7]=='b':
            shutil.copy2(drt + file,'./TestData/CMU/'+drt[9:13]+file)

# Processing saarburkcen

In [33]:
def merge(speech,egg):
    assert len(speech) == len(egg)
    merged = np.zeros((2,len(speech)))
    merged[0,:] = speech
    merged[1,:] = egg
    return merged
    

In [10]:
def save(num):
    for name in ['phrase','a_h','a_l','a_lhl','a_n','i_h','i_l','i_lhl','i_n','u_h','u_l','u_lhl','u_n']:
        if os.path.exists('./saarbrucken/export/%d-%s.wav'%(num,name)) and os.path.exists('./saarbrucken/export/%d-%s-egg.wav'%(num,name)):
            if not os.path.exists('./saarbrucken/merge/%d-%s.wav'%(num,name)):
                n_iau,sr = librosa.load('./saarbrucken/export/%d-%s.wav'%(num,name),sr=None)
                n_iau_egg,sr = librosa.load('./saarbrucken/export/%d-%s-egg.wav'%(num,name),sr=None)

                merged = merge(n_iau,n_iau_egg)
                merged = librosa.resample(merged,sr,16000)
                sf.write('./saarbrucken/merge/%d-%s.wav'%(num,name), merged.T,16000, format='WAV', endian='LITTLE', subtype='PCM_16')
            

In [24]:
df = pd.read_csv('./saarbrucken_metadata.csv',engine='python')
l = os.listdir('./saarbrucken/export/')
n=[]
for file in l:
    num = int(file.split('-')[0])
    if not num in n:
        n.append(num)

In [25]:
def f(v):
    if v in n:
        return True
    return False
df = df[df['ID'].apply(f)]

In [11]:
pool = Pool(64)
pool.map(save,n)
# for n in tqdm_notebook(range(num)):
#     save(n)

In [26]:
df_W = df[df.G == 'w']
df_M = df[df.G == 'm']

print(len(df_W),len(df_M))
print(len(df_W[df_W['T']=='n']),len(df_M[df_M['T'] =='n']))
print(len(df_W[df_W['T'] =='p']),len(df_M[df_M['T'] =='p']))

1153 889
426 261
727 628


In [31]:
normal = df[df['T'] == 'n']
Pathology = df[df['T'] == 'p']
# Pathology_w = Pathology[Pathology['G'] == 'w']
# Pathology_m = Pathology[Pathology['G'] == 'm']
l = os.listdir('./saarbrucken/merge')
os.makedirs('./TrainData/saarbrucken/',exist_ok=True)
os.makedirs('./TestData/saarbrucken/',exist_ok=True)
cnt1,cnt2 = 0,0
for num in tqdm_notebook(n) :
    if num in normal.ID.values:
        for name in ['phrase','a_h','a_l','a_lhl','a_n','i_h','i_l','i_lhl','i_n','u_h','u_l','u_lhl','u_n']:
            try:
                shutil.copy2('./saarbrucken/merge/%d-%s.wav'%(num,name),'./TrainData/saarbrucken/%d-%s.wav'%(num,name))
            except:
                pass
    elif num in Pathology.ID.values:
        if np.random.uniform()<0.5:
            cnt1 +=1
        for name in ['phrase','a_h','a_l','a_lhl','a_n','i_h','i_l','i_lhl','i_n','u_h','u_l','u_lhl','u_n']:
            try:
                shutil.copy2('./saarbrucken/merge/%d-%s.wav'%(num,name),'./TrainData/saarbrucken/%d-%s.wav'%(num,name))
            except:
                pass
        else:
            cnt2 +=1
        for name in ['phrase','a_h','a_l','a_lhl','a_n','i_h','i_l','i_lhl','i_n','u_h','u_l','u_lhl','u_n']:
            try:
                shutil.copy2('./saarbrucken/merge/%d-%s.wav'%(num,name),'./TestData/saarbrucken/%d-%s.wav'%(num,name))
            except:
                pass

print("Pathology %d Train, %d Test"%(cnt1,cnt2))

In [32]:
cnt1,cnt2

(660, 695)